# Problem 1

### 1. Calculate the coefficient according to the training set

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame,Series
from scipy import stats
from sklearn.linear_model import RidgeCV


data = pd.read_csv("climate_change_1.csv") 
train = data[data.Year<=2006]
train.head()




,Year,Month,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols,Temp
0,1983,5,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,0.109
1,1983,6,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,0.118
2,1983,7,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,0.137
3,1983,8,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,0.176
4,1983,9,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,0.149


In [33]:
def closed_form_1(x,y):
    return np.linalg.inv(x.T @ x) @ (x.T @ y)

x_train= train[['MEI','CO2','CH4','N2O','CFC-11','CFC-12','TSI','Aerosols']]
x_train_withc = x_train.copy()
x_train_withc["constant"] = 1
x_train_withc = x_train_withc.values
y_train = train["Temp"].values
theta1 = closed_form_1(x_train_withc,y_train) 
theta1_pd = pd.DataFrame(theta1)
theta1_pd.columns=["coefficient"]
theta1_pd.index=["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols","constant"]
theta1_pd


,coefficient
MEI,0.064205
CO2,0.006457
CH4,0.000124
N2O,-0.016528
CFC-11,-0.006630
CFC-12,0.003808
TSI,0.093141
Aerosols,-1.537613
constant,-124.594262


### 2.1 Calculate the $R^2$ of training set

The mathematical formula for the linear model is:
\begin{equation}
y_i = \beta_0 + \beta_1 x_{i1} +\beta_2 x_{i2}+……+\beta_n x_{in}+ \epsilon
\label{eq:}
\end{equation}
where i is the index of samples

$x_i$ is the expanatory variables, in this case, it means CO2, CH4, and other factors.

$y_i$ is the dependent variables, in this case, it means Temperature.

$\beta_0$ is y intercept

$\beta_i$ is coefficients for expanatory variables

$\epsilon$ is error term

in this case the formula of linear regression is :


Temp = 0.064205\*MEI+0.006457\*CO2+0.000124\*CH4+(-0.016528\)*N2O+(-0.006630)\*CFC-11+0.003808\*CFC-12+0.093141\*TSI+(-1.537613)*Aerosols-124.594262


And $R^2$ of traning set is 0.751 and $R^2$ of test set is 0.225

In [4]:

y_hat = x_train_withc @ theta1

y_bar = np.sum(y_train) / len (y_train)  # 求均值

ssreg = np.sum((y_hat -y_bar)**2)
sstot = np.sum((y_train - y_bar)**2)

r_square_test = ssreg / sstot
r_square_test

coefficient    0.750893
dtype: float64

### 2.2. Calculate the $R^2$ of testing set

In [27]:
test = data [data.Year > 2006]
x_test = test[["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols"]]
x_test_withc = x_test.copy()
x_test_withc["constant"] = 1 
x_test_withc = x_test_withc.values
y_test = test["Temp"].values
y_hat = x_test_withc @ theta1

y_bar = np.sum(y_test) / len (y_test)  

ssreg = np.sum((y_hat -y_bar)**2)
sstot = np.sum((y_test - y_bar)**2)

r_square_test = ssreg / sstot
r_square_test

coefficient    0.225177
dtype: float64

In [28]:
np.linalg.det(x_test.T @ x_test)

844849471.4178216

In [29]:
def r_square(x,y,theta):
    yhat = x @ theta
    ybar = np.sum(y) / len (y)  # 求均值
    ssreg = np.sum((yhat - ybar)**2)
    sstot = np.sum((y - yhat)**2) + ssreg
    return ssreg / sstot

### 3 significant  of variables

In [36]:
y_train_predict = x_train_withc @ theta1
MSE = (sum((y_train-y_train_predict)**2))/(len(x_train_withc)- np.shape(x_train_withc)[1])
var_b = MSE * (np.linalg.inv(np.dot(x_train_withc.T,x_train_withc)).diagonal())
svar_b = np.sqrt(var_b)
ts_b = theta1/ svar_b
p_values =[2*(1-stats.t.cdf(np.abs(i),(len(x_train_withc)-1))) for i in ts_b]
result = pd.DataFrame()
result["p value"] = p_values
result["t statistic"] = ts_b
result.index=["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols","constant"]
result

,p value,t statistic
MEI,0.000000e+00,9.923226
CO2,5.042595e-03,2.826420
CH4,8.101405e-01,0.240469
N2O,5.464020e-02,-1.929726
CFC-11,5.913566e-05,-4.077834
CFC-12,2.085760e-04,3.757293
TSI,1.057310e-09,6.312561
Aerosols,5.109246e-12,-7.210301
constant,1.381915e-09,-6.265174


### 4. climate_change_2.csv

In [37]:
data2 = pd.read_csv("climate_change_2.csv")
data2_train = data2[data2.Year <= 2006]
data2_test = data2[data2.Year > 2006]
data2.head()

,Year,Month,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols,NO,Temp
0,1983,5,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,2.63859,0.109
1,1983,6,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,2.63371,0.118
2,1983,7,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,2.63322,0.137
3,1983,8,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,2.63135,0.176
4,1983,9,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,2.64840,0.149


In [44]:
test2 = data2 [data2.Year > 2006]
x_test2 = test2[["CH4"]]
x_test2_withc = x_test2.copy()
x_test2_withc["constant"] = 1 
x_test2_withc = x_test2_withc.values
y_test2 = test2["NO"].values
theta2 = closed_form_1(x_test2_withc,y_test2) 
theta2

array([1.e-03, 1.e+00])

In [49]:
r = r_square(x_test2_withc,y_test2,theta2)
r

1.0

we can see, in climate2, the CH4 and NO is highly relate since the R square is 1.0. under this circumstance, the $X^T X$ is close to zero, so the data is irreversibility.

# Problem 2

###  1 Loss fuction of LASSO and Ridge regression

loss function for linear model with L1 regularization(LASSO):
\begin{equation}
\frac{1}{2m}[\sum_{i=1}^{m}(\hat{y}_i-y_i)^2+\lambda\sum_{j=1}^{n}\theta_j^2]
\label{eq:with L1 regularization}
\end{equation}
where m is the number of samples, n is the number of explanatory variables


loss function for linear model with L2 regularization (Ridge regression):
\begin{equation}
\frac{1}{2m}[\sum_{i=1}^{m}(\hat{y}_i-y_i)^2+\lambda\sum_{j=1}^{n}\left|\theta_j\right|]
\label{eq:with L2 regularization}
\end{equation}
where m is the number of samples, n is the number of explanatory variables


### 2 L2 regression

Suppose Lamda = 0.1

In [51]:
def closed_form_2(x,y,lamda):
    a = np.identity(np.shape(x)[1])
    a[np.shape(x)[1] - 1,np.shape(x)[1] - 1] = 0
    return np.linalg.inv(x.T @ x + lamda * a) @ (x.T @ y)

In [52]:
theta2 = closed_form_2(x_train_withc, y_train, 0.1) 
theta2_pd = pd.DataFrame(theta2)
theta2_pd.columns=["coefficient_with_L2"]
theta2_pd.index=["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols","constant"]
theta2_pd

,coefficient_with_L2
MEI,0.058361
CO2,0.007299
CH4,0.000189
N2O,-0.018100
CFC-11,-0.006900
CFC-12,0.003886
TSI,0.087290
Aerosols,-0.997495
constant,-116.507105


### 3 Compare linear regression and L2 

In [53]:
theta2_pd_compare = theta2_pd.copy()
theta2_pd_compare["coefficient_without_L2"] = theta1
theta2_pd_compare

,coefficient_with_L2,coefficient_without_L2
MEI,0.058361,0.064205
CO2,0.007299,0.006457
CH4,0.000189,0.000124
N2O,-0.018100,-0.016528
CFC-11,-0.006900,-0.006630
CFC-12,0.003886,0.003808
TSI,0.087290,0.093141
Aerosols,-0.997495,-1.537613
constant,-116.507105,-124.594262


The reason that L2 is more robust is the higer $R^2$ .
According to the result in 4 and result in Problem 1, the $R^2$ on test set is higher than the model without L2 regularization.

### 4. cross validation

In [60]:
lamda = [10, 1, 0.1, 0.01, 0.001]
r_square_train_list = []
r_square_test_list = []
for i in lamda:
    theta = closed_form_2(x_train_withc, y_train, i) 
    r_square_train = r_square(x_train_withc, y_train, theta)
    r_square_train_list.append(r_square_train)
    r_square_test = r_square(x_test_withc, y_test, theta)
    r_square_test_list.append(r_square_test)
r_result = pd.DataFrame()
r_result["r_square of train_set"] = r_square_train_list
r_result["r_square of test_set"] = r_square_test_list
r_result.index = lamda
r_result

,r_square of train_set,r_square of test_set
10.000,0.699675,0.299372
1.000,0.713193,0.264121
0.100,0.739433,0.228466
0.010,0.749615,0.217094
0.001,0.750766,0.216300


In [61]:
ridgecv = RidgeCV(alphas=[10, 1, 0.1, 0.01, 0.001], cv=5)
ridgecv.fit(x_train, y_train)
print("when cv = 5, the best lambda is {}".format(ridgecv.alpha_))
ridgecv = RidgeCV(alphas=[100,10, 1, 0.1, 0.01, 0.001,0.0001], cv=10)
ridgecv.fit(x_train, y_train)
print("when cv = 10, the best lambda is {}".format(ridgecv.alpha_))

when cv = 5, the best lambda is 0.001
when cv = 10, the best lambda is 0.01


# Problem 3

Step one:
    
    get the correlation coefficient matrix and calculate 
$vif_i=\sum_{j\not=i}\frac{1}{1-R_j^2}$ 
    
    to find the variable that is highly correlated with other variables.

In [71]:
correlation = x_train.corr()
correlation

,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols
MEI,1.000000,-0.041147,-0.033419,-0.050820,0.069000,0.008286,-0.154492,0.340238
CO2,-0.041147,1.000000,0.877280,0.976720,0.514060,0.852690,0.177429,-0.356155
CH4,-0.033419,0.877280,1.000000,0.899839,0.779904,0.963616,0.245528,-0.267809
N2O,-0.050820,0.976720,0.899839,1.000000,0.522477,0.867931,0.199757,-0.337055
CFC-11,0.069000,0.514060,0.779904,0.522477,1.000000,0.868985,0.272046,-0.043921
CFC-12,0.008286,0.852690,0.963616,0.867931,0.868985,1.000000,0.255303,-0.225131
TSI,-0.154492,0.177429,0.245528,0.199757,0.272046,0.255303,1.000000,0.052117
Aerosols,0.340238,-0.356155,-0.267809,-0.337055,-0.043921,-0.225131,0.052117,1.000000


In [72]:
def vif(x):
    correlation = x.corr()
    vif = np.zeros(np.shape(x)[1])
    for i in range(np.shape(x)[1]):
        for j in range(np.shape(x)[1]):
            if i!= j :
                vif[i] += 1 / (1-correlation.iloc[i,j] ** 2)
    return vif


In [74]:
vif(x_train)

array([ 7.16562416, 34.27380297, 29.28798479, 35.58693396, 12.45783187,
       28.92174134,  7.31504007,  7.53966644])

Step two:
    
    N2O has the biggest vif.
    remove N2O since it has biggest vif

In [76]:
x_train_1 = x_train.drop(columns = ["N2O"])
vif(x_train_1)

array([ 6.16303483, 12.54336315, 24.03285936, 11.08234907, 24.86817211,
        6.27347894,  6.41150023])

Step three:
    
    CFC-12 has the biggest vif.
    remove CFC-12 since it has biggest vif

In [77]:
x_train_1 = x_train_1.drop(columns = ["CFC-12"])
vif(x_train_1)

array([ 5.16296617,  8.87929113, 10.03582924,  6.99846198,  5.20375483,
        5.35811013])

Step four:
    
    CH4 has the biggest vif.
    remove CH4 since it has biggest vif

In [79]:
x_train_1 = x_train_1.drop(columns = ["CH4"])
vif(x_train_1)

array([4.16184807, 4.53864495, 4.44581178, 4.13960328, 4.28084691])

Step five:

   CFC-11 has the biggest vif.
   remove CFC-11 since it has biggest vif

In [81]:
x_train_1 = x_train_1.drop(columns = ["CFC-11"])
vif(x_train_1)

array([3.15706424, 3.17947391, 3.05967918, 3.27891411])

In [88]:
x_train_1.corr()

,MEI,CO2,TSI,Aerosols
MEI,1.000000,-0.041147,-0.154492,0.340238
CO2,-0.041147,1.000000,0.177429,-0.356155
TSI,-0.154492,0.177429,1.000000,0.052117
Aerosols,0.340238,-0.356155,0.052117,1.000000


In [89]:
x_train_withc_1 =  x_train_1.copy()
x_train_withc_1["constant"] = 1
x_test_1 = x_test.drop(columns = ["N2O","CH4","CFC-12","CFC-11"])
x_test_withc_1 =  x_test_1.copy()
x_test_withc_1["constant"] = 1
theta1 = closed_form_1(x_train_withc_1, y_train) 
r_square_test = r_square(x_test_withc_1, y_test, theta1)
r_square_test

0.32569067160230875

# Problem 4

In [62]:
def Gradient_Descent(alpha, theta_0, x, y, tol):
    theta = theta_0
    cost = (1/len(x)) * (x.T @ (x @ theta_0 - y))
    count = 0
    while not ((np.all(cost) <= tol) | (count > 10000)):
        count += 1
        theta = theta - alpha * (1./len(x)) * (x.T @ (x @ theta - y))
        cost = (1./len(x)) * (x.T @ (x @ theta - y))
    print('times is {}'.format(count))
    return theta


In [68]:
x_GD = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],[2, 5, 7, 8, 6, 10, 13, 17, 18, 19]])
x_GD = np.insert(x_GD, 0, np.ones(10), axis = 0).T
y_GD = np.array([10, 33, 34, 37, 36, 39, 41, 44, 44, 45])
theta_GD = Gradient_Descent(0.01, np.ones(3), x_GD, y_GD, 1e-5)

times is 10001


In [69]:
theta_GD

array([21.04187218,  2.06418162,  0.37191704])

In [70]:

theta3 = closed_form_1(x_GD,y_GD) 
theta3

array([21.0418722 ,  2.06418161,  0.37191704])

the parameter we calculate through grandient is almost the same with the linear regression!